In [5]:
import numpy as np
from sklearn.datasets import load_iris

In [15]:
iris = load_iris()
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [28]:
# the input size is the amount of variables in the dataset
# in datascience terms, this can be seen as the amount of independent variables
# the output size equals the amount of classes you want to predict
# in this case, we have 3 classes, meaning 3 output nodes
input_size = len(iris['data'][0])
output_size = len(np.unique(iris['target']))
hidden_size = 4
print(f"input size: {input_size}, hidden size: {hidden_size}, output size: {output_size}")


input size: 4, hidden size: 4, output size: 3


In [26]:

def init_network(input_size, hidden_size, output_size):
  W_IH = np.random.randn(input_size, hidden_size) * np.sqrt(2.0 / input_size)
  W_HO = np.random.randn(hidden_size, output_size) * np.sqrt(2.0 / hidden_size)
  b_H = np.zeros(hidden_size)
  b_O = np.zeros(output_size)


In [29]:
# relu is the activation function used for the network
# it works by only using positive values, and setting all negative values to 0
def relu(x):
  return np.maximum(0, x)

def relu_der(x):
    return (x > 0).astype(float)

In [30]:
class NeuralNetwork:

  # the weights are a matrix (can be seen as nested lists)
  # the first dimension is the amount of nodes in the previous layer
  # the second dimension is the amount of nodes in the next layer
  # the bias is a vector (like a single list) of zeros, which will be updated during backpropagation
  def __init__(self, input_size, hidden_size, output_size):
    self.W_IH = np.random.randn(input_size, hidden_size) * np.sqrt(2.0 / input_size)
    self.W_HO = np.random.randn(hidden_size, output_size) * np.sqrt(2.0 / hidden_size)
    self.b_H = np.zeros(hidden_size)
    self.b_O = np.zeros(output_size)
  
  def feedforward(self, x):
    self.hidden_z = np.dot(x, self.W_IH) + self.b_H
    self.hidden_activation = relu(self.hidden_z)

    self.output_z = np.dot(self.hidden_activation, self.W_HO) + self.b_O
    self.output_activation = relu(self.output_z)
    return self.output_activation
  
  def cost(self, y_hat, y):
    return np.mean((y_hat - y)**2)
  
  def cost_der(self, y_hat, y):
    return 2 / y.shape[0] * (y_hat - y)
  
  def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / np.sum(exp_x, keepdims=True)
  
  def softmax_cross_entropy_derivative(y_pred, y_true):
    return y_pred - y_true
  
  # 
  def backpropagation(self, y_hat, y, learning_rate):
    Eo = self.cost_der(y_hat, y) * relu_der(self.output_z)
    Eh = np.dot(Eo, self.W_HO.T) * relu_der(self.hidden_z)
    
    # Update weights and biases
    self.W_HO -= learning_rate * np.dot(self.hidden_activation.T, Eo)
    self.b_O -= learning_rate * np.sum(Eo, axis=0)
    self.W_IH -= learning_rate * np.dot(x.T, Eh)
    self.b_H -= learning_rate * np.sum(Eh, axis=0)
    




SyntaxError: incomplete input (3894861064.py, line 26)

In [32]:
# Define the Test class
class Test:
    def __init__(self):
        self.test = 1

    def poop(self):
        print(self.test)

# Create an instance of the Test class
test_instance = Test()

# Call the poop method
test_instance.poop()

1
